<a href="https://colab.research.google.com/github/aaronlwan/deep-learning-stock-lows/blob/main/Deep_Learning_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import Modules
!pip install yfinance
import pandas as pd
import numpy as np
import yfinance as yf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import math
from tensorflow.python.framework import ops
import matplotlib.pyplot as plt

# Data Processing

### Normalize Data

In [ ]:
# Normalize Data Within Window (min-max)
def normalize_in_window(data):
  new_data = []
  for j in range(len(data)):
    prices = []
    volumes = []
    for i in range(len(data[j])):
      if (i + 1) % 5 == 0:
        volumes.append(data[j][i])
      else:
        prices.append(data[j][i])
    maxprice = max(prices)
    minprice = min(prices)
    maxvol = max(volumes)
    minvol = min(volumes)
    newrow = []
    price_index = 0
    vol_index = 0
    for k in range(len(data[j])):
      if (k + 1) % 5 == 0:
        newrow.append((volumes[vol_index] - minvol)/(maxvol-minvol))
        vol_index += 1
      else:
        newrow.append((prices[price_index] - minprice)/(maxprice - minprice))
        price_index += 1
    newrow.append(maxprice)
    newrow.append(minprice)
    new_data.append(newrow)
  data = np.array(new_data)
  return data

# Normalize Over the Training Timeframe (min-max)
def normalize_all(data, train_window):
  prices = []
  volumes = []
  for j in range(train_window):
    for i in range(len(data[j])):
      if (i + 1)%5 == 0:
        volumes.append(data[j][i])
      else:
        prices.append(data[j][i])
  maxprice = max(prices)
  minprice = min(prices)
  minvol = min(volumes)
  maxvol = max(volumes)
  print('Max Price:', maxprice)
  print('Min Price:', minprice)
  new_data = []
  for k in range(len(data)):
    row = []
    for i in range(len(data[k])):
      if (i + 1)%5 == 0:
        row.append((data[k][i]-minvol)/(maxvol-minvol))
      else:
        row.append((data[k][i]-minprice)/(maxprice-minprice))
    new_data.append(row)
  return np.array(new_data)

# Standardize over the whole Training Timeframe (0 mean, 1std)
def standardize_all(data):
  prices = []
  volumes = []
  for j in range(len(data)):
    for i in range(len(data[j])):
      if (i + 1)%5 == 0:
        volumes.append(data[j][i])
      else:
        prices.append(data[j][i])
  meanprice = np.average(np.array(prices))
  stdprice = np.std(np.array(prices))
  meanvol = np.sum(np.array(volumes))/len(prices)
  stdvol = np.std(np.array(volumes))
  print('Mean Price:', meanprice)
  print('STD Price:', stdprice)
  new_data = []
  for k in range(len(data)):
    row = []
    for i in range(len(data[k])):
      if (i + 1)%5 == 0:
        row.append((data[k][i]-meanvol)/(stdvol))
      else:
        row.append((data[k][i]-meanprice)/(stdprice))
    new_data.append(row)
  return np.array(new_data)

def log_normalization(data):
  return np.log(data)

### yfinance Data Processsing

In [ ]:
# Get Data From Yfinance
def getData(ticker, input_interval, input_candles, output_interval):

  # Load Input Data
  # Input Data Columns: Date, Open, High, Low, Close, Volume
  input = yf.download(ticker, interval=input_interval)
  input = np.array(pd.DataFrame({'Date': input.index, 'Open': input.values[:, 0], 'High': input.values[:, 1],
                         'Low': input.values[:, 2], 'Close': input.values[:, 3],
                         'Volume': input.values[:, 5]}))
  # Output Data Colums: Date, Low
  # Load Output Data
  output = yf.download(ticker, interval=output_interval)
  output = np.array(pd.DataFrame({'Date': output.index, 'Low': output.values[:, 2]}))
  
  # Match Input w/ Output
  # Matched Data Columns: Repeat (Date, Open, High, Low, Close, Volume) for each input candle, Output Low
  data = []
  input_dates = list(input[:, 0])
  for row in output:
    data_row = []
    outdate = row[0]
    try:
      input_index = input_dates.index(outdate)
      # Collect the input_candles preceeding the outdate
      for i in range(input_candles):
        candle = input[input_index - 1 - i][1:]
        for item in candle:
          data_row.append(item)
      data_row.append(row[1])
      data.append(data_row)
    except: 
      pass
  return np.array(data)

In [ ]:
# Get Data
data = getData("SPY", "1d", 10, "1d")
print(data.shape)
data = normalize_all(data, 7000)

### Intraday Data Processing

In [ ]:
# Intraday Process (5min to Predict 1hr)
input_data = np.array(pd.read_csv('/content/drive/MyDrive/SPY_qjrt28/5min.csv', header=None))
output_data = np.array(pd.read_csv('/content/drive/MyDrive/SPY_qjrt28/1hr.csv', header=None))
data = []
deleted = []
i = 0
output_data = output_data[1:]
index = 0
while True:
  try:
    # Select 24 candles of data
    selection = input_data[i: i+24][:, 1:]
    last_timestamp = input_data[i: i+24][:, 0][23]
    # Check last candle, if it doesn't match up we just omit from dataset b/c we have a lot of datapoints
    if int(last_timestamp[14:16]) == 55:
      selection = list(selection.flatten())
      selection.append(output_data[index][1])
      data.append(selection)
      i += 12
    else:
      deleted.append(index)
      difference = int(last_timestamp[14:16]) + 60 - 55
      i += int(12 - difference/5)
    index += 1
  except:
    break

data = np.array(data)
print(data.shape)
print(deleted)
data = normalize_all(data, 33000)

In [ ]:
# 1hr to predict 1d
from pandas import Timestamp
input_data = np.array(pd.read_csv('/content/drive/MyDrive/SPY_qjrt28/1hrinput.csv', header=None))
output = yf.download('SPY', interval='1d')
output_data = np.array(pd.DataFrame({'Date': output.index, 'Low': output.values[:, 2]}))
outdates = list(output_data[:, 0])
start = outdates.index(Timestamp('2005-01-05 00:00:00'))
output_data = output_data[start:]
data = []
deleted = []
i = 0
index = 0
while True:
  try:
    # Select 24 candles of data
    selection = input_data[i: i+24][:, 1:]
    last_timestamp = input_data[i: i+24][:, 0][23]
    # Check last candle, if it doesn't match up we just omit from dataset b/c we have a lot of datapoints
    if int(last_timestamp[11:13]) == 19:
      selection = list(selection.flatten())
      selection.append(output_data[index][1])
      data.append(selection)
      i += 12
    else:
      deleted.append(index)
      difference = int(last_timestamp[11:13]) - 7
      i += int(12 - difference)
    index += 1
  except:
    break
data = np.array(data)
print(data.shape)
data = log_normalization(data)

In [ ]:
#1hr to predict 1hr
input_data = np.array(pd.read_csv('/content/drive/MyDrive/SPY_qjrt28/1hrinputandoutput.csv', header=None))
output_data = input_data[:, 3]
data = []
index = 0
output_data = output_data[10:]
for i in range(len(output_data)):
  row = input_data[i: i+10, 1:]
  row = row.flatten()
  row = list(row)
  row.append(output_data[i])
  data.append(row)
data = np.array(data)
print(data.shape)
data = normalize_all(data, 66000)

### Split Data

In [ ]:
# Split Train/Test based on time frame (simulates predicting the future using past data)
x = data[:, [i for i in range(50)]]
y = np.transpose([data[:, 50]])
X_train, X_test, Y_train, Y_test = np.transpose(x[:66000]), np.transpose(x[66000:]), np.transpose(y[:66000]), np.transpose(y[66000:])

# Single Layer RNN

In [ ]:
# Create Mini-Batches
def random_mini_batches(X, Y, mini_batch_size=64):
  mini_batches = []
  m = X.shape[1]
  num_complete_mini_batches = math.floor(m/mini_batch_size)
  for k in range(num_complete_mini_batches):
    batch_X = X[:, k*mini_batch_size: (k+1)*mini_batch_size]
    batch_Y = Y[:, k*mini_batch_size: (k+1)*mini_batch_size]
    mini_batches.append((batch_X, batch_Y))
  if m % mini_batch_size != 0:
    batch_X = X[:, mini_batch_size * num_complete_mini_batches:]
    batch_Y = Y[:, mini_batch_size * num_complete_mini_batches:]
    mini_batches.append((batch_X, batch_Y))
  return mini_batches

In [ ]:
def create_placeholders(n_x, n_y):
  X = tf.placeholder(tf.float32, shape=[n_x, None], name='Placeholder_1')
  Y = tf.placeholder(tf.float32, shape=[n_y, None], name='Placeholder_2')
  h_prev = tf.placeholder(tf.float32, [8, 1], name='h_prev')
  return X, Y, h_prev


In [ ]:
X, Y, h_prev = create_placeholders(8, 1)
print ("X = " + str(X))
print ("Y = " + str(Y))
print(h_prev)

In [ ]:
import tensorflow as tf2

def initialize_parameters(parameter_size, hidden_size):
  Wa = tf.get_variable("Wa", [hidden_size, parameter_size], initializer = tf2.initializers.GlorotNormal())
  Wya = tf.get_variable("Wya", [1, hidden_size], initializer = tf2.initializers.GlorotNormal())
  Waa = tf.get_variable("Waa", [hidden_size, hidden_size], initializer = tf2.initializers.GlorotNormal())

 

  ba = tf.get_variable("ba", [hidden_size, 1], initializer=tf.zeros_initializer())
  by = tf.get_variable("by", [1, 1], initializer=tf.zeros_initializer())
  parameters = {
      "Wa": Wa,
      "Wya": Wya,
      "Waa": Waa,
      "ba": ba,
      "by": by,
  }
  return parameters

In [ ]:
def forward_propagation(X, parameters, h_prev, max_batch):
  Wa = parameters["Wa"]
  Wya = parameters["Wya"]
  Waa = parameters["Waa"]
  ba = parameters["ba"]
  by = parameters["by"]
 
  a, y = {}, []
  a[-1] = h_prev
  for t in range(max_batch):
    try:
      a[t] = tf.nn.relu(tf.add(tf.add(ba, tf.matmul(Waa, a[t - 1])), tf.matmul(Wa, X[:, t:t+1])))
      yt = tf.nn.relu(tf.add(tf.matmul(Wya, a[t]), by))
      y.append(yt)
    except:
      pass
  return y, a

In [ ]:
def compute_cost(predictions, Y):
  cost = tf.reduce_mean(tf.math.abs(tf.subtract(predictions, Y)))
  return cost

In [ ]:
def model(X_train, Y_train, X_test, Y_test, learning_rate = 0.0001, num_epochs = 500, print_cost = True, minibatch_size=16):
  ops.reset_default_graph()
  (n_x, m) = X_train.shape
  n_y = Y_train.shape[0]
  costs = []
  
  X, Y, h_prev = create_placeholders(n_x, n_y)
  parameters = initialize_parameters(n_x, 8)
  preds, h = forward_propagation(X, parameters, h_prev, minibatch_size)
  cost = compute_cost(preds, Y)
  optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
  init = tf.global_variables_initializer()

  print("Beginning Training")
  with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
      hprevious = np.zeros((8, 1))
      epoch_cost = 0                      # Defines a cost related to an epoch
      num_minibatches = int(m / minibatch_size) # number of minibatches of size minibatch_size in the train set
      minibatches = random_mini_batches(X_train, Y_train, minibatch_size)
      for i in range(num_minibatches):
          # Select a minibatch
          (minibatch_X, minibatch_Y) = minibatches[i]
          _, minibatch_cost, h_new = sess.run([optimizer, cost, h], feed_dict={X: minibatch_X, Y: minibatch_Y, h_prev: hprevious})
          epoch_cost += minibatch_cost
          hprevious = h_new[list(h.keys())[-1]]
      epoch_cost/=num_minibatches

      # Print the cost every epoch
      if print_cost == True and epoch % 100 == 0:
          print ("Cost after epoch %i: %f" % (epoch, epoch_cost))
      if print_cost == True and epoch % 5 == 0:
          costs.append(epoch_cost)

    
    # plot the cost
    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations (per fives)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()

    # Save parameters
    parameters = sess.run(parameters)
    print ("Parameters have been trained!")

    
    return parameters, hprevious

In [ ]:
parameters, h = model(X_train, Y_train, X_test, Y_test)

In [ ]:
def compute_accuracy(X, Y, parameters, hprevious):
  Wa = parameters["Wa"]
  Wya = parameters["Wya"]
  Waa = parameters["Waa"]
  ba = parameters["ba"]
  by = parameters["by"]
  a, y = {}, []
  a[-1] = hprevious
  for t in range(X.shape[1]):
    a[t] = np.maximum(np.add(np.add(ba, np.matmul(Waa, a[t - 1])), np.matmul(Wa, X[:, t:t+1])), 0)
    yt = np.maximum(np.add(np.matmul(Wya, a[t]), by), 0)
    y.append(yt)


  error = 0
  for i in range(len(y)):
    error += abs(y[i] - Y[0][i])
  print('MAE on Normalized Data', error/len(y))


  # Convert prediction back into actual prediction
  mre = 0
  for i in range(len(y)):
    maxprice = 450.6300048828125
    minprice = 42.8125
    actual_predict = y[i]*(maxprice - minprice) + minprice
    true_price = Y[0][i]*(maxprice - minprice) + minprice
    #print(actual_predict, true_price)
    mre += abs(actual_predict - true_price)/(true_price)
  print('MRE on Actual Prices:', mre[0][0]/len(y))

compute_accuracy(X_test, Y_test, parameters, h)

# Two Layer RNN

In [ ]:
def create_placeholders(n_x, n_y):
  X = tf.placeholder(tf.float32, shape=[n_x, None], name='Placeholder_1')
  Y = tf.placeholder(tf.float32, shape=[n_y, None], name='Placeholder_2')
  h_prev = tf.placeholder(tf.float32, [8, 1], name='h_prev')
  h2_prev = tf.placeholder(tf.float32, [4, 1], name='h2_prev')
  return X, Y, h_prev, h2_prev


In [ ]:
X, Y, h_prev, h2_prev = create_placeholders(4, 1)
print ("X = " + str(X))
print ("Y = " + str(Y))
print(h_prev)
print(h2_prev)

In [ ]:
import tensorflow as tf2

def initialize_parameters(parameter_size, hidden_size):
  Wa = tf.get_variable("Wa", [hidden_size, parameter_size], initializer = tf2.initializers.GlorotNormal())
  Wya = tf.get_variable("Wya", [hidden_size, hidden_size], initializer = tf2.initializers.GlorotNormal())
  Waa = tf.get_variable("Waa", [hidden_size, hidden_size], initializer = tf2.initializers.GlorotNormal())

  Wa2 = tf.get_variable("Wa2", [4, hidden_size], initializer = tf2.initializers.GlorotNormal())
  Wya2 = tf.get_variable("Wya2", [1, 4], initializer = tf2.initializers.GlorotNormal())
  Waa2 = tf.get_variable("Waa2", [4, 4], initializer = tf2.initializers.GlorotNormal())

  ba = tf.get_variable("ba", [hidden_size, 1], initializer=tf.zeros_initializer())
  ba2 = tf.get_variable("ba2", [4, 1], initializer=tf.zeros_initializer())
  by = tf.get_variable("by", [hidden_size, 1], initializer=tf.zeros_initializer())
  by2 = tf.get_variable("by2", [1, 1], initializer=tf.zeros_initializer())
  parameters = {
      "Wa": Wa,
      "Wya": Wya,
      "Waa": Waa,
      "ba": ba,
      "by": by,
      "Wa2": Wa2,
      "Wya2": Wya2,
      "Waa2": Waa2,
      "ba2": ba2,
      "by2": by2
  }
  return parameters

In [ ]:
def forward_propagation(X, parameters, h_prev, h2_prev, max_batch):
  Wa = parameters["Wa"]
  Wya = parameters["Wya"]
  Waa = parameters["Waa"]
  ba = parameters["ba"]
  by = parameters["by"]
  Wa2 = parameters["Wa2"]
  Wya2 = parameters["Wya2"]
  Waa2 = parameters["Waa2"]
  ba2 = parameters["ba2"]
  by2 = parameters["by2"]
  a, a2, y = {}, {}, []
  a[-1] = h_prev
  a2[-1] = h2_prev
  for t in range(max_batch):
    try:
      a[t] = tf.nn.relu(tf.add(tf.add(ba, tf.matmul(Waa, a[t - 1])), tf.matmul(Wa, X[:, t:t+1])))
      yt = tf.nn.relu(tf.add(tf.matmul(Wya, a[t]), by))
      a2[t] = tf.nn.relu(tf.add(tf.add(ba2, tf.matmul(Waa2, a2[t - 1])), tf.matmul(Wa2, yt)))
      yt2 = tf.nn.relu(tf.add(tf.matmul(Wya2, a2[t]), by2))
      y.append(yt2)
    except:
      pass
  return y, a, a2

In [ ]:
def compute_cost(predictions, Y):
  cost = tf.reduce_mean(tf.math.abs(tf.subtract(predictions, Y)))
  return cost

In [ ]:
def model(X_train, Y_train, X_test, Y_test, learning_rate = 0.0001, num_epochs = 1000, print_cost = True, minibatch_size=8):
  ops.reset_default_graph()
  (n_x, m) = X_train.shape
  n_y = Y_train.shape[0]
  costs = []
  
  X, Y, h_prev, h2_prev = create_placeholders(n_x, n_y)
  parameters = initialize_parameters(n_x, 8)
  preds, h, h2 = forward_propagation(X, parameters, h_prev, h2_prev, minibatch_size)
  cost = compute_cost(preds, Y)
  optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
  init = tf.global_variables_initializer()

  print("Beginning Training")
  with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
      hprevious = np.zeros((8, 1))
      h2previous = np.zeros((4, 1))
      epoch_cost = 0                      # Defines a cost related to an epoch
      num_minibatches = int(m / minibatch_size) # number of minibatches of size minibatch_size in the train set
      minibatches = random_mini_batches(X_train, Y_train, minibatch_size)
      for i in range(num_minibatches):
          # Select a minibatch
          (minibatch_X, minibatch_Y) = minibatches[i]
          _, minibatch_cost, h_new, h2_new = sess.run([optimizer, cost, h, h2], feed_dict={X: minibatch_X, Y: minibatch_Y, h_prev: hprevious, h2_prev: h2previous})
          epoch_cost += minibatch_cost
          hprevious = h_new[list(h.keys())[-1]]
          h2previous = h2_new[list(h.keys())[-1]]
      epoch_cost/=num_minibatches

      # Print the cost every epoch
      if print_cost == True and epoch % 100 == 0:
          print ("Cost after epoch %i: %f" % (epoch, epoch_cost))
      if print_cost == True and epoch % 5 == 0:
          costs.append(epoch_cost)

    
    # plot the cost
    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations (per fives)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()

    # Save parameters
    parameters = sess.run(parameters)
    print ("Parameters have been trained!")

    
    
    return parameters, hprevious, h2previous

In [ ]:
parameters, h, h2 = model(X_train, Y_train, X_test, Y_test)

In [ ]:
def compute_accuracy(X, Y, parameters, hprevious, h2previous):
  Wa = parameters["Wa"]
  Wya = parameters["Wya"]
  Waa = parameters["Waa"]
  ba = parameters["ba"]
  by = parameters["by"]
  Wa2 = parameters["Wa2"]
  Wya2 = parameters["Wya2"]
  Waa2 = parameters["Waa2"]
  ba2 = parameters["ba2"]
  by2 = parameters["by2"]
  a, a2, y = {}, {}, []
  a[-1] = hprevious
  a2[-1] = h2previous
  for t in range(X.shape[1]):
    a[t] = np.maximum(np.add(np.add(ba, np.matmul(Waa, a[t - 1])), np.matmul(Wa, X[:, t:t+1])), 0)
    yt = np.maximum(np.add(np.matmul(Wya, a[t]), by), 0)
    a2[t] = np.maximum(np.add(np.add(ba2, np.matmul(Waa2, a2[t - 1])), np.matmul(Wa2, yt)), 0)
    yt2 = np.maximum(np.add(np.matmul(Wya2, a2[t]), by2), 0)
    y.append(yt2)


  error = 0
  for i in range(len(y)):
    error += abs(y[i] - Y[0][i])
  print('MAE on Normalized Data', error/len(y))


  # Convert prediction back into actual prediction
  mre = 0
  for i in range(len(y)):
    maxprice = 429.6400146484375
    minprice = 42.8125
    actual_predict = y[i]*(maxprice - minprice) + minprice
    true_price = Y[0][i]*(maxprice - minprice) + minprice
    #print(actual_predict, true_price)
    mre += abs(actual_predict - true_price)/(true_price)
  print('MRE on Actual Prices:', mre[0][0]/len(y))

compute_accuracy(X_test, Y_test, parameters, h, h2)

# CNN

In [ ]:
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, Conv1D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, RNN
from keras.callbacks import EarlyStopping
from keras.models import Model, load_model
from keras.initializers import glorot_uniform, glorot_normal

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

In [ ]:
# Format X data to fit in CNN
# Format: m sets of 5xdx1 matricies, where d is the number of candles in the window
def format(X, d):
  # Each window is a column of X
  new_X = []
  for i in range(len(X[0])):
    col = X[:, i]
    sample = np.zeros((5, 1))
    # Every 5 datapoints in the column is one column in the sample
    bar_data = []
    for j in range(len(col)):
      bar_data.append(col[j])
      if (j + 1) % 5 == 0:
        sample = np.concatenate((sample, np.transpose([np.array(bar_data)])), axis=1)
        bar_data = []
    sample = np.delete(sample, 0, axis=1)
    sample = np.expand_dims(sample, 2)
    new_X.append(sample)
  return np.array(new_X)
X_train = format(X_train, 10)
print(X_train.shape)
Y_train = np.squeeze(Y_train)

In [ ]:
from keras.layers.convolutional import Conv3D
from keras.layers.recurrent import SimpleRNN
from keras.layers import ConvLSTM2D
from keras.layers import Reshape

def basic_cnn(input_shape = (5, 10, 1)):
  tf.compat.v1.disable_eager_execution()
  X_input = Input(input_shape)
  X = Conv2D(filters=5, kernel_size=(3, 3), padding='valid', kernel_initializer=glorot_normal())(X_input)
  X = Conv2D(filters=5, kernel_size=(3, 3), padding='valid', kernel_initializer=glorot_normal())(X_input)
  X = Flatten()(X)
  X = Dense(128, activation='relu', kernel_initializer=glorot_normal())(X)
  X = Dense(64, activation='relu', kernel_initializer=glorot_normal())(X)
  X = Dense(32, activation='relu', kernel_initializer=glorot_normal())(X)
  X = Dense(16, activation='relu', kernel_initializer=glorot_normal())(X)
  X = Dense(1, activation='relu', kernel_initializer=glorot_normal())(X)
  model = Model(inputs = X_input, outputs = X, name='BasicCNN')
  return model

In [ ]:
X_test = format(X_test, 10)
Y_test = np.squeeze(Y_test)

In [ ]:
model = basic_cnn()
#es = EarlyStopping(monitor='val_loss', mode='min', patience=50)
model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs = 100, batch_size = 32)

In [ ]:
preds = model.evaluate(X_test, Y_test)
predictions = model.predict(X_test)
print('Average Error', preds[0])

# Convert prediction back into actual prediction
mre = 0
pred_range = len(predictions)
for i in range(pred_range):
  maxprice = 450.094
  minprice = 52.0695
  meanprice = 179.1915384746307
  stdprice = 101.32692381699914
  actual_predict = predictions[i]*(maxprice - minprice) + minprice
  true_price = Y_test[i]*(maxprice - minprice) + minprice
  #actual_predict = predictions[i]*stdprice + meanprice
  #true_price = Y_test[i]*stdprice + meanprice
  mre += abs(actual_predict - true_price)/(true_price)
  #print(actual_predict, true_price)
print('MRE on Actual Prices:', mre[0]/pred_range)

# CNN-RNN

In [ ]:
# We want X to have shape m x 5 x d x 1
# where m is total samples, d is number of days in each window, and 5 is the 
# number of features in each day
def cnn_rnn_format_x(X, d):
  # Each window is a column of X
  new_X = []
  for i in range(len(X[0])):
    col = X[:, i]
    sample = np.zeros((5, 1))
    # Every 5 datapoints in the column is one column in the sample
    bar_data = []
    for j in range(len(col)):
      bar_data.append(col[j])
      if (j + 1) % 5 == 0:
        sample = np.concatenate((sample, np.transpose([np.array(bar_data)])), axis=1)
        bar_data = []
    sample = np.delete(sample, 0, axis=1)
    sample = np.expand_dims(sample, 2)
    new_X.append(sample)
  return np.array(new_X)

print(cnn_rnn_format_x(X_train, 10).shape)
X_train = cnn_rnn_format_x(X_train, 10)

In [ ]:
from keras.layers import Input, Dense, LSTM, Flatten, TimeDistributed, Conv2D, Reshape, SimpleRNN, GRU
from keras import Sequential
from keras.initializers import glorot_normal, glorot_uniform, HeNormal, HeUniform
tf.compat.v1.disable_eager_execution()
model = Sequential()
model.add(Conv2D(filters=3, kernel_size=(2, 3), strides=1, padding='valid', kernel_initializer=glorot_normal(), input_shape=(5, 10, 1)))
model.add(Conv2D(filters=2, kernel_size=(2, 3), strides=1, padding='valid', kernel_initializer=glorot_normal()))
model.add(Reshape((4, 9)))
model.add(SimpleRNN(20, activation='relu', kernel_initializer=glorot_normal()))
model.add(Dense(1, activation='relu', kernel_initializer=glorot_normal()))
model.compile(optimizer='adam', loss='mean_absolute_error')
model.summary()

In [ ]:
Y_train = np.squeeze(Y_train)
X_test = cnn_rnn_format_x(X_test, 10)
Y_test = np.squeeze(Y_test)

In [ ]:
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs = 100, batch_size = 32)

In [ ]:
preds = model.evaluate(X_test, Y_test)
predictions = model.predict(X_test)
print('Mean Error', preds)

# Convert prediction back into actual prediction
mre = 0
overpredict = 0
for i in range(len(predictions)):
  maxprice = 450.094
  minprice = 52.0695
  meanprice = 179.1915384746307
  stdprice = 101.09464575990093
  actual_predict = predictions[i]*(maxprice - minprice) + minprice
  true_price = Y_test[i]*(maxprice - minprice) + minprice
  #actual_predict = np.exp(predictions[i])
  #true_price = np.exp(Y_test[i])
  
  #actual_predict = predictions[i]*stdprice + meanprice
  #true_price = Y_test[i]*stdprice + meanprice
  if actual_predict > true_price:
    overpredict += 1
  mre += abs(actual_predict - true_price)/(true_price)
  #print(actual_predict, true_price)
print('MRE on Actual Prices:', mre[0]/len(predictions))
print('Overpredict Rate', overpredict/len(predictions))

In [ ]:
# Simulation: buy a share at projected low if price reaches the projected low, then sell at close
output_data = np.array(pd.read_csv('/content/drive/MyDrive/SPY_qjrt28/1hrinputandoutput.csv', header=None))
removed = 0
output_data = output_data[15:]
output_data = output_data[66000:]
#print(len(output_data), len(predictions))
print('For the test interval, the price of SPY changed from', output_data[0][4], 'to', output_data[len(output_data)-1][4])

maxprice = 450.094
minprice = 52.0695
acc_value = 1000
winners = 0
buys = 0

for i in range(len(predictions)):
  actual_predict = predictions[i]*(maxprice - minprice) + minprice
  true_low = output_data[i][3]
  true_close = output_data[i][4]
  #print(actual_predict, true_low)
  # 0.9942 is a "correction" constant to account for the model's error
  if true_low < actual_predict*0.9942:
    acc_value = acc_value - actual_predict*0.9942+ true_close
    if actual_predict*0.9942 <= true_close:
      winners += 1
    buys += 1
    #print('Bought at:', str(actual_predict*0.9942, ' Sold at:', str(true_close), ' Current Acc Value:', str(acc_value))
print('This current strategy produces', str((acc_value[0]/1000 - 1)*100)+ "% Profit over the test timeframe")
print('Winrate:', winners/buys)
print('Total Transactions:', buys)